In [1]:
!pip install azure-cognitiveservices-vision-customvision

distutils: /Users/masato/miniforge3/include/python3.9/UNKNOWN
sysconfig: /Users/masato/miniforge3/include/python3.9
user = False
home = None
root = None
prefix = None
distutils: /Users/masato/miniforge3/include/python3.9/UNKNOWN
sysconfig: /Users/masato/miniforge3/include/python3.9
user = False
home = None
root = None
prefix = None


In [3]:
import glob
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

In [44]:
base_url = 'https://fishclassification0814.cognitiveservices.azure.com/'
projectID = '6404afa5-b2d7-4ca7-90f5-a4b6ec673a71'
publish_iteration_name = 'Iteration1'
prediction_key = '043d083540e24f4d9bf22320683763c0'

In [45]:
def testModel(testfiles, fishname):
    data_count = len(testfiles)
    true_count = 0

    # 予測用インスタンスの作成　…①
    prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
    predictor = CustomVisionPredictionClient(base_url, prediction_credentials)
    for testfile in testfiles:
        predicts = {}
        with open(testfile, mode='rb') as f:
            # 予測実行　…②
            results = predictor.classify_image(projectID, publish_iteration_name, f.read())

        # 予測結果のタグの数だけループ　…③
        for prediction in results.predictions:
            # 予測した魚とその確率を紐づけて格納　…④
            predicts[prediction.tag_name] = prediction.probability

        # 一番確率の高い魚を予測結果として選択　…⑤
        prediction_result = max(predicts, key=predicts.get)

        # 予測結果が合っていれば正解数を増やす
        if fishname == prediction_result:
            true_count += 1

    # 正解率の算出
    accuracy = (true_count / data_count) * 100
    print('正解率:' + str(accuracy) + '%')


In [47]:
#検証用画像を保存したルートディレクトリパス
root_dir = 'fish_images/'
#検証対象の魚一覧
fishnames = ['アイゴ', 'オニカサゴ']

for fishname in fishnames:
    print('*****' + fishname + '*****')
    #testデータのリストを取得
    testfiles = glob.glob(root_dir + fishname + '/test/*')
    testModel(testfiles, fishname)

*****アイゴ*****


CustomVisionErrorException: Operation returned an invalid status code 'Access Denied'

In [41]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid